# Pull FRED Series data via FredSeries

This documentation serves as a quick start guide to `FredSeries`. To review all of the `FredSeries` functionality please see the [API documentation](https://pyfredapi.readthedocs.io/en/latest/references/api.html#module-pyfredapi.api.series).

A primary use case of the FRED web service is to pull series observations for analysis or reporting. `pyfredapi` provides the `FredSeries` class to make pull series easy.

In [1]:
from pyfredapi import FredSeries
from rich import print as rprint

In [2]:
client = FredSeries()

## Get series info

Before pulling series data, it is helpful to understand the `pyfredapi`'s `get_series_info` method and `SeriesInfo` object. Each series published by FRED has associated metadata, such as the observations start and end dates, the frequency at which the series is published, and so on. You can query a series' information directly with `get_series_info`. `get_series_info` returns a `SeriesInfo` object that contains all the metadata for the given series.

In the below example, we request information on the Feds Funds Rate. From the result, we can see that the Federal Funds Rate series is available back to 1954-07-01 and is published every month.

In [3]:
fed_funds_info = client.get_series_info(series_id="FEDFUNDS")

In [4]:
# Using rich to pretty print the SeriesInfo
rprint(fed_funds_info)

SeriesInfo(
    id='FEDFUNDS',
    realtime_start='2022-09-27',
    realtime_end='2022-09-27',
    title='Federal Funds Effective Rate',
    observation_start='1954-07-01',
    observation_end='2022-08-01',
    frequency='Monthly',
    frequency_short='M',
    units='Percent',
    units_short='%',
    seasonal_adjustment='Not Seasonally Adjusted',
    seasonal_adjustment_short='NSA',
    last_updated='2022-09-01 15:21:01-05',
    popularity=97,
    notes='Averages of daily figures. \n\nFor additional historical federal funds rate data, please see  Daily 
Federal Funds Rate from 1928-1954 (https://fred.stlouisfed.org/categories/33951).\n\nThe federal funds rate is the 
interest rate at which depository institutions trade federal funds (balances held at Federal Reserve Banks) with 
each other overnight. When a depository institution has surplus balances in its reserve account, it lends to other 
banks in need of larger balances. In simpler terms, a bank with excess cash, which is often referred to as 
liquidity, will lend to another bank that needs to quickly raise liquidity. (1) The rate that the borrowing 
institution pays to the lending institution is determined between the two banks; the weighted average rate for all 
of these types of negotiations is called the effective federal funds rate.(2) The effective federal funds rate is 
essentially determined by the market but is influenced by the Federal Reserve through open market operations to 
reach the federal funds rate target.(2)\nThe Federal Open Market Committee (FOMC) meets eight times a year to 
determine the federal funds target rate. As previously stated, this rate influences the effective federal funds 
rate through open market operations or by buying and selling of government bonds (government debt).(2) More 
specifically, the Federal Reserve decreases liquidity by selling government bonds, thereby raising the federal 
funds rate because banks have less liquidity to trade with other banks. Similarly, the Federal Reserve can increase
liquidity by buying government bonds, decreasing the federal funds rate because banks have excess liquidity for 
trade. Whether the Federal Reserve wants to buy or sell bonds depends on the state of the economy. If the FOMC 
believes the economy is growing too fast and inflation pressures are inconsistent with the dual mandate of the 
Federal Reserve, the Committee may set a higher federal funds rate target to temper economic activity. In the 
opposing scenario, the FOMC may set a lower federal funds rate target to spur greater economic activity. Therefore,
the FOMC must observe the current state of the economy to determine the best course of monetary policy that will 
maximize economic growth while adhering to the dual mandate set forth by Congress. In making its monetary policy 
decisions, the FOMC considers a wealth of economic data, such as: trends in prices and wages, employment, consumer 
spending and income, business investments, and foreign exchange markets.\nThe federal funds rate is the central 
interest rate in the U.S. financial market. It influences other interest rates such as the prime rate, which is the
rate banks charge their customers with higher credit ratings. Additionally, the federal funds rate indirectly 
influences longer- term interest rates such as mortgages, loans, and savings, all of which are very important to 
consumer wealth and confidence.(2)\nReferences\n(1) Federal Reserve Bank of New York. "Federal funds." Fedpoints, 
August 2007.\n(2) Board of Governors of the Federal Reserve System. "Monetary Policy 
(https://www.federalreserve.gov/monetarypolicy.htm)".'
)

`SeriesInfo` also provides a useful method to open the FRED webpage for the series. Call `open_url` on the info object and a new browser tab will open.

In [5]:
fed_funds_info.open_url()

## Pull data with `FredSeries`

`FredSeries` provides a few methods for pull data from FRED.

* `get_series` - get the latest series data
* `get_series_all_releases` - get data for all releases
* `get_series_initial_release` - get data for the initial release
* `get_series_asof_date` - get data released on or before a specific date

Each of these methods returns a `SeriesData` object. SeriesData is made of two components. The first is info, which is the same as the above. The second is the data itself. The data can returned as with a pandas dataframe or json.

In [6]:
fed_funds_data = client.get_series(series_id="FEDFUNDS")

In [7]:
fed_funds_data.info == fed_funds_info

True

In [8]:
fed_funds_data.data.tail()

,realtime_start,realtime_end,date,value
813,2022-09-27,2022-09-27,2022-04-01,0.33
814,2022-09-27,2022-09-27,2022-05-01,0.77
815,2022-09-27,2022-09-27,2022-06-01,1.21
816,2022-09-27,2022-09-27,2022-07-01,1.68
817,2022-09-27,2022-09-27,2022-08-01,2.33


In [9]:
fed_funds_data.data.dtypes

realtime_start            object
realtime_end              object
date              datetime64[ns]
value                    float64
dtype: object